### Advent of Code 

In [59]:
# Includes (Some includes inspired by Peter Norvig solution (https://github.com/norvig/pytudes))

import os
import urllib.request


def Inputstr(day, year=2017): 
    "The contents of this day's input file as a str."
    return Input(day, year).read().rstrip('\n')

def Input(day, year=2017):
    "Open the day's input file"
    directory = 'data/advent{}/day{}/'.format(year, day)
    filename = directory + 'input{}.txt'.format(day)
    return open(filename)

def mapt(fn, *arg):
    return tuple(map(fn, *arg))

def Atom(input):
    "Parse a str token into a number, or leave it as a str."
    try:
        return int(input)
    except ValueError:
        try:
            return float(token)
        except ValueError:
            return token
    
def Vector(lines):
    return mapt(Atom, lines.replace(',', ' ').split())

def Array(input):
    lines = input.splitlines()
    return mapt(Vector, lines)


In [73]:
assert mapt(int, '1234') == (1, 2, 3, 4)

assert Array('''5 1 9 5
7 5 3
2 4 6 8''') == (
                (5, 1, 9, 5), 
                (7, 5, 3),
                (2, 4, 6, 8)
               )



### --- Day 1: Inverse Captcha ---

The captcha requires you to review a sequence of digits (your puzzle input) and find the sum of all digits that match the next digit in the list. The list is circular, so the digit after the last digit is the first digit in the list.

For example:

1122 produces a sum of 3 (1 + 2) because the first digit (1) matches the second digit and the third digit (2) matches the fourth digit.
1111 produces 4 because each digit (all 1) matches the next.
1234 produces 0 because no digit matches the next.
91212129 produces 9 because the only digit that matches the next one is the last digit, 9.



In [42]:
def inverse_captcha(input):
    d = input[0]
    n = len(input)
    s = 0
    for ad in input[1:]:
        if ad == d:
            s += (ord(ad) - 48)
        d = ad
        
    if input[n-1] == input[0]:
        s += (ord(input[n-1]) - 48)
    
    return s

input = Inputstr(1)
N = len(input)

print (inverse_captcha(input))

digits = mapt(int, input)
sum(digits[i] for i in range(N) if digits[i] == digits[i - 1])


1097


1097

#### Part 2

Now, instead of considering the next digit, it wants you to consider the digit halfway around the circular list. That is, if your list contains 10 items, only include a digit in your sum if the digit 10/2 = 5 steps forward matches it. Fortunately, your list has an even number of elements.


In [48]:
def inverse_captcha_halfway(input):
    n = len(input)
    h = int (n / 2)
    the_sum = 0
    for i in range(n):
        if input[i] == input[ (i+h) % n]:
            the_sum += (ord(input[i]) - 48)
    
    return the_sum


input = Inputstr(1)
N = len(input)

print (inverse_captcha_halfway(input))

# Norvig version
digits = mapt(int, input)

sum(digits[i] for i in range(N) if digits[i] == digits[ i - N // 2])

1188


1188

### Day 2: Checksum

In [89]:
d = Array(Inputstr(2))

sum(max(t) - min(t) for t in d)

51139